# LAB 07: Scale free networks, configuration model, and degree-preserving randomization

August 9th 2022

* This is a python tutorial on scale free networks, and the cofiguration model.
* There is no marking for this tutorial. You do not need to submit your solution to us, but this exercise will help you to start working on your project
* In the begining of this tutorial you will find a similar code than used in the class. In the end, there are exercises for you.
* We recommend you to solve this lab until next Tuesday (August 16th).

## Scale-free networks

* Load the protein.edgelist network from http://networksciencebook.com/translations/en/resources/networks.zip
* What is the maximum degree of the network?
* Compare it to the expected largest hub size according to the power law distribution
* Classify your network into:  'anomalous regime', 'ultra small world', and 'small world'.
* Compute the average distance of the network.


In [ ]:
import networkx as nx
import powerlaw
import numpy as np
import scipy

# Load the network

G = nx.read_edgelist("protein.edgelist.txt")
N = G.number_of_nodes()
degrees = (np.array(G.degree())[:,1]).astype(int)

max_degree = max(degrees)

# Fit power law degree distribution

fit = powerlaw.Fit(degrees, discrete=True)

# Estimate the maximun hub size according to the power law
kmax = fit.xmin*(N**(1/(fit.alpha-1)))

print(max_degree, kmax)

# Classifying the network 'regime'
def regime(gamma):
    if gamma <= 2:
        return 'anomalous'
    if gamma < 3:
        return 'ultra-small world'
    if gamma == 3:
        return 'critical point'
    return 'small world'

regime(fit.alpha)

# Computing the average shortest distance of the largest component
components = [component for component in nx.connected_components(G)]
sizes = [len(component) for component in components]
avg_dist = [nx.average_shortest_path_length(G.subgraph(C)) 
            for C in components if len(C) == max(sizes)]
            

## Configuration model


* Create a procedure that generates graphs with a given degree sequence
* Given an alpha, generates 100 graphs with power lawer degree distribution
* Compare the expected size of the largest hub with the largest degree of the generated networks.


In [ ]:
import random

random.seed(0)

def configuration_model(degrees):

    stubs = []
    for i in range(len(degrees)):
        for j in range(degrees[i]):
            stubs.append(i)

    G = nx.Graph()

    while stubs != []:
        i = random.sample(stubs, 1)[0]
        stubs.remove(i)
        j = random.sample(stubs, 1)[0]
        stubs.remove(j)
        G.add_edge(i,j)
    return G

G = configuration_model([2,1,1])
nx.draw(G)

nx.convert_matrix.to_numpy_array(G)


# Power law degree distribution for a given gamma
gamma = 2.8
zeta = scipy.special.zeta(gamma)
N = 100

p = [(k**(-gamma))/zeta for k in range(1,N)]

p = p/sum(p) # making sure it sums 1.

# Generate random graphs

M = 100
graphs = []
for i in range(M):
    sample = [1]
    # Generate degree sequences according to the power law distribution
    while (sum(sample) % 2) != 0:
        sample = np.random.choice(np.array(range(1, N)), size=N, p=p)

    graphs.append(configuration_model(sample))
    #graphs.append(nx.configuration_model(sample))
    
kmax = (N**(1/(fit.alpha-1)))
max_degree = [max(np.array(G.degree())[:,1].astype(int)) for G in graphs]

print(kmax, np.mean(max_degree))



## Exercise 

* Using the data from your project: (1) obtain the maximum degree of the network, (2) compare it to the expected size according to the power law degree distribution, and (3) compute the average distance of the network. Is it an ultra-small world, small world or anomalous network?
* Given a network, create a random procedure that returns a new random network preserving its degrees.

In [ ]:
# Procedure that preserves the orginal degrees

import networkx as nx
import numpy as np
import random

G = nx.barabasi_albert_graph(50, 1)
G_orig = G.copy()
degrees = (np.array(G_orig.degree())[:,1]).astype(int)
edges = G_orig.edges

for edge in edges:
    if G.has_edge(*edge):
      v1 = edge[0]
      v2 = edge[1]
      possible_edges = [e for e in G.edges if set(e) != set(edge) and not G.has_edge(v1, e[1])
                      and not G.has_edge(e[0], v2)]
      sel_edge = random.sample(possible_edges, 1)[0]
      G.remove_edge(v1, v2)
      G.remove_edge(sel_edge[0], sel_edge[1])
      G.add_edge(v1, sel_edge[1])
      G.add_edge(sel_edge[0], v2)
    
#nx.draw(G_orig)
#nx.draw(G)

degrees2 = (np.array(G.degree())[:,1]).astype(int)

degrees-degrees2



array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])